In [1]:
import pandas as pd
import numpy as np

In [3]:
# Read in the data
INPUT_DIR = './m5-forecasting-accuracy'
df_sale = pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv')
df_calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv')
df_price = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv')
df_sample = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')

In [4]:
# Forecast with MA
df_last_sale = df_sale.iloc[:, np.r_[0,-28:0]].melt('id', var_name='d', value_name='sale')
df_last_sale = df_last_sale.merge(df_calendar.loc[:,['d','date','wday']])
last_date = int(df_last_sale.d.max()[2:])
df_last_sale = df_last_sale.groupby(['id','wday'])['sale'].mean()


df_ma_forecast = df_sample.copy()
df_ma_forecast.columns = ['id'] + ['d_' + str(last_date + x) for x in range(1, 29)]
df_ma_forecast = df_ma_forecast.loc[df_ma_forecast.id.str.contains('validation')]

df_ma_forecast = df_ma_forecast.melt('id', var_name='d', value_name='sale')
df_ma_forecast = df_ma_forecast.drop('sale',axis = 1)
df_ma_forecast = df_ma_forecast.merge(df_calendar.loc[:,['d','date','wday']])
df_ma_forecast = df_ma_forecast.join(df_last_sale, on=['id', 'wday'])

df_ma_forecast = df_ma_forecast.pivot(index='id', columns='d', values='sale')
df_ma_forecast = df_ma_forecast.reset_index()

In [8]:
df_ma_forecast.to_csv("./submission_ma.csv")